# Lab 4: Build a Custom MCP Server Using Azure Functions

In this lab, you'll create a custom remote **Model Context Protocol (MCP)** server using Azure Functions. The MCP server uses the Azure Functions MCP server extension to provide tools for AI models, agents, and assistants.

We'll take the functions from **Lab 1** (data analysis) and expose them as MCP tools that can be consumed by GitHub Copilot and other AI agents.

## What is MCP?

The **Model Context Protocol (MCP)** is an open standard that enables AI models to securely interact with external tools and data sources. By creating an MCP server, you can:

- 🔧 Expose custom tools to AI assistants like GitHub Copilot
- 🔒 Maintain secure access control with function keys
- 🚀 Scale automatically with serverless Azure Functions
- 🔄 Enable end-to-end integration between AI and your business logic

## Architecture Overview

```
┌─────────────────┐     ┌──────────────────────┐     ┌─────────────────┐
│  GitHub Copilot │────▶│  Azure Functions     │────▶│  Your Business  │
│  (MCP Client)   │◀────│  (MCP Server)        │◀────│  Logic          │
└─────────────────┘     └──────────────────────┘     └─────────────────┘
                              │
                              ▼
                        MCP Tools:
                        - analyze_data
                        - hello_mcp
                        - save_snippet
                        - get_snippets
```

## Prerequisites

- ✅ Python 3.11+
- ✅ Visual Studio Code with these extensions:
  - Azure Functions extension
  - Azure Developer CLI extension
  - GitHub Copilot
- ✅ Azurite storage emulator
- ✅ Azure CLI
- ✅ Azure Developer CLI (`azd`)
- ✅ An Azure account with an active subscription

## Step 1: Install Required Tools

First, let's verify you have all the required tools installed.

In [1]:
import subprocess
import shutil


def check_tool(name: str, command: list) -> bool:
    """Check if a tool is installed and accessible."""
    try:
        result = subprocess.run(command, capture_output=True, text=True, timeout=10)
        version = result.stdout.strip() or result.stderr.strip()
        print(f"✅ {name}: {version.split(chr(10))[0]}")
        return True
    except (subprocess.SubprocessError, FileNotFoundError):
        print(f"❌ {name}: Not installed")
        return False


print("Checking required tools...\n")

tools = [
    ("Python", ["python", "--version"]),
    ("Azure CLI", ["az", "--version"]),
    ("Azure Developer CLI", ["azd", "version"]),
    ("Azure Functions Core Tools", ["func", "--version"]),
]

all_installed = all(check_tool(name, cmd) for name, cmd in tools)

if all_installed:
    print("\n🎉 All tools are installed! You're ready to proceed.")
else:
    print("\n⚠️ Some tools are missing. Please install them before continuing.")

Checking required tools...

✅ Python: Python 3.11.14
✅ Azure CLI: azure-cli                         2.82.0
✅ Azure Developer CLI: azd version 1.23.0 (commit 9a02add5930c2c87b6b2c78443e4d8132587417d)
✅ Azure Functions Core Tools: 4.6.0

🎉 All tools are installed! You're ready to proceed.


### Installation Commands (if needed)

If any tools are missing, use these commands to install them:

```bash
# Azure CLI (macOS)
brew install azure-cli

# Azure Developer CLI
brew install azd

# Azure Functions Core Tools (requires Azure tap)
brew tap azure/functions
brew install azure-functions-core-tools@4

# Azurite (npm package for storage emulation)
npm install -g azurite
```


## Step 2: Initialize the MCP Server Project

We'll use the Azure Developer CLI (`azd`) to create a project from the official MCP server template.

### Use the Official Template 

Run this command in your terminal to create a new MCP server project:

```bash
# Navigate to the tool registry directory
cd src/tool_registry/mcps

# Initialize from the official template
# ⚠️ Replace <your-name> with YOUR unique identifier (e.g., your name or NetID)
azd init --template remote-mcp-functions-python -e mcp-<your-name>
```

The `-e` flag sets an environment name that will be used for deployment. **Use a unique name** like `mcp-johndoe` or `mcp-jsmith42` to avoid conflicts with other students.

> **Note**: The project has already been created at `src/tool_registry/mcps/` in this repository. You can skip the `azd init` step, but make sure to set your unique environment name before deploying with `azd env new mcp-<your-name>`.

## Step 3: Understanding the MCP Server Code Structure

An MCP server built on Azure Functions consists of **one core file**: `function_app.py`. This file defines "tools" that AI agents can invoke via the Model Context Protocol.

### Key Concepts

| Concept | Description |
|---------|-------------|
| **`@app.generic_trigger`** | Decorator that registers a Python function as an MCP tool |
| **`type="mcpToolTrigger"`** | Specifies this is an MCP tool (not HTTP, Timer, etc.) |
| **`toolName`** | The name AI agents use to invoke this tool |
| **`toolProperties`** | JSON schema defining the tool's input parameters |
| **`description`** | Natural language description helping AI decide when to use this tool |

### How It Works

```
AI Agent (e.g., Copilot)          MCP Server (Azure Function)
         │                                    │
         │  1. "Analyze [1,2,3,4,5]"          │
         │ ─────────────────────────────────► │
         │                                    │  2. Parse request
         │                                    │  3. Execute analyze_data()
         │  4. {"count":5, "mean":3, ...}     │
         │ ◄───────────────────────────────── │
         │                                    │
```

The `toolProperties` JSON schema tells the AI what parameters it can pass. For example:
```json
[{"propertyName": "values", "propertyType": "string", "description": "JSON array of numbers"}]
```

Now let's examine the actual code:

In [2]:
# MCP Server Function Code
# This is the main function_app.py that defines our MCP tools

function_app_code = '''
"""MCP Server Azure Functions - Lab 4

This module defines MCP tools that can be consumed by AI agents like GitHub Copilot.
We're exposing the data analysis functionality from Lab 1 as MCP tools.
"""

import json
import logging
import azure.functions as func
from typing import Any, Dict, List

app = func.FunctionApp(http_auth_level=func.AuthLevel.FUNCTION)

# ============================================================================
# MCP Tool Properties Definitions
# ============================================================================

class ToolProperty:
    """Defines a property for an MCP tool."""
    
    def __init__(self, name: str, property_type: str, description: str):
        self.name = name
        self.property_type = property_type
        self.description = description
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            "propertyName": self.name,
            "propertyType": self.property_type,
            "description": self.description
        }


# Tool properties for analyze_data
tool_properties_analyze_data = [
    ToolProperty("values", "string", "A JSON array of numbers to analyze, e.g., [1, 2, 3, 4, 5]")
]
tool_properties_analyze_data_json = json.dumps([p.to_dict() for p in tool_properties_analyze_data])

# Tool properties for save_snippet
tool_properties_save_snippet = [
    ToolProperty("snippet_name", "string", "The name to save the snippet under"),
    ToolProperty("snippet_content", "string", "The code snippet content to save")
]
tool_properties_save_snippet_json = json.dumps([p.to_dict() for p in tool_properties_save_snippet])

# Tool properties for get_snippet
tool_properties_get_snippet = [
    ToolProperty("snippet_name", "string", "The name of the snippet to retrieve")
]
tool_properties_get_snippet_json = json.dumps([p.to_dict() for p in tool_properties_get_snippet])


# ============================================================================
# MCP Tool: hello_mcp
# ============================================================================

@app.generic_trigger(
    arg_name="context",
    type="mcpToolTrigger",
    toolName="hello_mcp",
    description="A simple hello world tool to verify MCP server connectivity.",
    toolProperties="[]"
)
def hello_mcp(context) -> str:
    """
    A simple hello world MCP tool.
    Returns a greeting message to verify the MCP server is working.
    """
    logging.info("hello_mcp tool invoked")
    return json.dumps({
        "message": "Hello from MCP Server! 👋",
        "status": "success",
        "lab": "Lab 4 - MCP Server with Azure Functions"
    })


# ============================================================================
# MCP Tool: analyze_data (from Lab 1)
# ============================================================================

@app.generic_trigger(
    arg_name="context",
    type="mcpToolTrigger",
    toolName="analyze_data",
    description="Analyze a list of numbers and return statistics including count, sum, mean, min, and max.",
    toolProperties=tool_properties_analyze_data_json
)
def analyze_data(context) -> str:
    """
    Analyze numerical data - the same functionality from Lab 1!
    
    This tool receives a list of numbers and returns statistical analysis.
    """
    logging.info("analyze_data MCP tool invoked")
    
    try:
        content = json.loads(context)
        values_str = content["arguments"].get("values", "[]")
        
        # Parse the values (they come as a JSON string)
        values = json.loads(values_str) if isinstance(values_str, str) else values_str
        
        if not values:
            return json.dumps({"error": "No values provided"})
        
        # Perform the analysis (same as Lab 1)
        result = {
            "count": len(values),
            "sum": sum(values),
            "mean": sum(values) / len(values),
            "min": min(values),
            "max": max(values),
            "values": values
        }
        
        logging.info(f"Analysis complete: {result}")
        return json.dumps(result)
        
    except Exception as e:
        logging.error(f"Error in analyze_data: {str(e)}")
        return json.dumps({"error": str(e)})


# ============================================================================
# MCP Tool: save_snippet
# ============================================================================

# In-memory storage for snippets (in production, use Azure Blob Storage)
_snippets_storage: Dict[str, str] = {}

@app.generic_trigger(
    arg_name="context",
    type="mcpToolTrigger",
    toolName="save_snippet",
    description="Save a code snippet with a given name for later retrieval.",
    toolProperties=tool_properties_save_snippet_json
)
def save_snippet(context) -> str:
    """
    Save a code snippet to storage.
    """
    logging.info("save_snippet MCP tool invoked")
    
    try:
        content = json.loads(context)
        snippet_name = content["arguments"].get("snippet_name", "")
        snippet_content = content["arguments"].get("snippet_content", "")
        
        if not snippet_name:
            return json.dumps({"error": "No snippet name provided"})
        
        if not snippet_content:
            return json.dumps({"error": "No snippet content provided"})
        
        # Save the snippet
        _snippets_storage[snippet_name] = snippet_content
        
        logging.info(f"Saved snippet: {snippet_name}")
        return json.dumps({
            "message": f"Snippet '{snippet_name}' saved successfully",
            "snippet_name": snippet_name,
            "status": "success"
        })
        
    except Exception as e:
        logging.error(f"Error in save_snippet: {str(e)}")
        return json.dumps({"error": str(e)})


# ============================================================================
# MCP Tool: get_snippet
# ============================================================================

@app.generic_trigger(
    arg_name="context",
    type="mcpToolTrigger",
    toolName="get_snippet",
    description="Retrieve a previously saved code snippet by name.",
    toolProperties=tool_properties_get_snippet_json
)
def get_snippet(context) -> str:
    """
    Retrieve a code snippet from storage.
    """
    logging.info("get_snippet MCP tool invoked")
    
    try:
        content = json.loads(context)
        snippet_name = content["arguments"].get("snippet_name", "")
        
        if not snippet_name:
            return json.dumps({"error": "No snippet name provided"})
        
        if snippet_name not in _snippets_storage:
            return json.dumps({
                "error": f"Snippet '{snippet_name}' not found",
                "available_snippets": list(_snippets_storage.keys())
            })
        
        snippet_content = _snippets_storage[snippet_name]
        
        logging.info(f"Retrieved snippet: {snippet_name}")
        return json.dumps({
            "snippet_name": snippet_name,
            "content": snippet_content,
            "status": "success"
        })
        
    except Exception as e:
        logging.error(f"Error in get_snippet: {str(e)}")
        return json.dumps({"error": str(e)})


# ============================================================================
# MCP Tool: list_snippets
# ============================================================================

@app.generic_trigger(
    arg_name="context",
    type="mcpToolTrigger",
    toolName="list_snippets",
    description="List all saved code snippets.",
    toolProperties="[]"
)
def list_snippets(context) -> str:
    """
    List all saved snippets.
    """
    logging.info("list_snippets MCP tool invoked")
    
    return json.dumps({
        "snippets": list(_snippets_storage.keys()),
        "count": len(_snippets_storage),
        "status": "success"
    })
'''

print("📝 MCP Server Function Code (function_app.py):")
print("=" * 60)
print(function_app_code[:2000] + "...\n[truncated for display]")

📝 MCP Server Function Code (function_app.py):

"""MCP Server Azure Functions - Lab 4

This module defines MCP tools that can be consumed by AI agents like GitHub Copilot.
We're exposing the data analysis functionality from Lab 1 as MCP tools.
"""

import json
import logging
import azure.functions as func
from typing import Any, Dict, List

app = func.FunctionApp(http_auth_level=func.AuthLevel.FUNCTION)

# ============================================================================
# MCP Tool Properties Definitions
# ============================================================================

class ToolProperty:
    """Defines a property for an MCP tool."""

    def __init__(self, name: str, property_type: str, description: str):
        self.name = name
        self.property_type = property_type
        self.description = description

    def to_dict(self) -> Dict[str, Any]:
        return {
            "propertyName": self.name,
            "propertyType": self.property_type,
       

### Save the Function Code

In [6]:
from pathlib import Path

PROJECT_ROOT = Path("../src/tool_registry/mcptest")

In [7]:
# Save the function_app.py file
function_app_path = PROJECT_ROOT / "src" / "function_app.py"

with open(function_app_path, "w") as f:
    f.write(function_app_code.strip())

print(f"✅ Saved: {function_app_path}")

✅ Saved: ../src/tool_registry/mcptest/src/function_app.py


## Step 4: Configuration Files

Azure Functions requires three configuration files. Understanding these is essential for debugging deployment issues.

### File Structure
```
src/tool_registry/mcps/
├── src/
│   ├── function_app.py      # Your MCP tools (Python code)
│   ├── host.json            # Azure Functions runtime config
│   ├── local.settings.json  # Local dev settings (secrets, connection strings)
│   └── requirements.txt     # Python dependencies
├── infra/                   # Bicep/ARM templates for Azure deployment
└── azure.yaml               # Azure Developer CLI config
```

### Configuration Details

| File | Purpose | Key Settings |
|------|---------|--------------|
| **host.json** | Runtime behavior | Extension bundles, logging, MCP enablement |
| **local.settings.json** | Local secrets | Storage connection, runtime version (⚠️ gitignored) |
| **requirements.txt** | Python deps | `azure-functions` (minimal for MCP) |

In [8]:
# host.json - Azure Functions host configuration
host_json = """
{
  "version": "2.0",
  "logging": {
    "applicationInsights": {
      "samplingSettings": {
        "isEnabled": true,
        "excludedTypes": "Request"
      }
    }
  },
  "extensionBundle": {
    "id": "Microsoft.Azure.Functions.ExtensionBundle",
    "version": "[4.*, 5.0.0)"
  },
  "extensions": {
    "mcp": {
      "enabled": true
    }
  }
}
"""

# local.settings.json - Local development settings
local_settings_json = """
{
  "IsEncrypted": false,
  "Values": {
    "FUNCTIONS_WORKER_RUNTIME": "python",
    "AzureWebJobsStorage": "UseDevelopmentStorage=true",
    "AzureWebJobsFeatureFlags": "EnableWorkerIndexing"
  }
}
"""

# requirements.txt - Python dependencies
requirements_txt = """
azure-functions
"""

# Save configuration files
configs = [
    (PROJECT_ROOT / "host.json", host_json),
    (PROJECT_ROOT / "local.settings.json", local_settings_json),
    (PROJECT_ROOT / "requirements.txt", requirements_txt),
]

for path, content in configs:
    with open(path, "w") as f:
        f.write(content.strip())
    print(f"✅ Saved: {path}")

print("\n🎉 All configuration files created!")

✅ Saved: ../src/tool_registry/mcptest/host.json
✅ Saved: ../src/tool_registry/mcptest/local.settings.json
✅ Saved: ../src/tool_registry/mcptest/requirements.txt

🎉 All configuration files created!


## Step 5: Start the Azurite Storage Emulator

**Why Azurite?** Azure Functions requires a storage account for internal state management (triggers, bindings, checkpoints). Azurite emulates Azure Blob, Queue, and Table storage locally.

### Architecture
```
┌─────────────────────────────────────────────────────────┐
│                    Local Development                     │
├─────────────────────────────────────────────────────────┤
│                                                          │
│   Azure Functions         Azurite (Storage Emulator)    │
│   ┌─────────────┐        ┌─────────────────────────┐   │
│   │ func start  │◄──────►│ Blob  :10000            │   │
│   │             │        │ Queue :10001            │   │
│   │ Port: 7071  │        │ Table :10002            │   │
│   └─────────────┘        └─────────────────────────┘   │
│                                                          │
└─────────────────────────────────────────────────────────┘
```

### Start Azurite

**Option A: VS Code Command Palette**
1. Press `F1` → Search: **Azurite: Start**

**Option B: Terminal**
```bash
azurite --silent --location /tmp/azurite --debug /tmp/azurite/debug.log
```

In [10]:
# Verify Azurite is running by checking if the blob service port is open
import socket


def check_port(host: str, port: int) -> bool:
    """Check if a port is open."""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(2)
        result = sock.connect_ex((host, port))
        sock.close()
        return result == 0
    except socket.error:
        return False


azurite_ports = [
    ("Blob Service", 10000),
    ("Queue Service", 10001),
    ("Table Service", 10002),
]

print("Checking Azurite services...\n")
all_running = True

for service, port in azurite_ports:
    if check_port("127.0.0.1", port):
        print(f"✅ {service}: Running on port {port}")
    else:
        print(f"❌ {service}: Not running on port {port}")
        all_running = False

if all_running:
    print("\n🎉 Azurite is running! You can proceed to start the MCP server.")
else:
    print("\n⚠️ Please start Azurite before continuing (F1 → 'Azurite: Start')")

Checking Azurite services...

✅ Blob Service: Running on port 10000
✅ Queue Service: Running on port 10001
✅ Table Service: Running on port 10002

🎉 Azurite is running! You can proceed to start the MCP server.


## Step 6: Run Your MCP Server Locally

The Azure Functions Core Tools (`func`) hosts your MCP server locally, simulating the Azure runtime.

### Start the Server

```bash
cd src/tool_registry/mcps/src
func start
```

### Expected Output
```
Azure Functions Core Tools (4.x)

Functions:
        hello_mcp: mcpToolTrigger
        analyze_data: mcpToolTrigger
        save_snippet: mcpToolTrigger
        get_snippet: mcpToolTrigger

MCP Server endpoint: http://localhost:7071/runtime/webhooks/mcp
```

### What's Happening Under the Hood

1. **Function Discovery**: Core Tools scans `function_app.py` for decorated functions
2. **MCP Extension**: The `mcpToolTrigger` bindings register tools with the MCP protocol handler
3. **SSE Endpoint**: The `/runtime/webhooks/mcp` endpoint serves Server-Sent Events for real-time communication
4. **Tool Invocation**: When an AI agent calls a tool, the request flows through the MCP protocol to your Python function

In [11]:
# MCP Server Endpoint Configuration
MCP_LOCAL_ENDPOINT = "http://localhost:7071/runtime/webhooks/mcp"

print(f"📡 Local MCP Server Endpoint:")
print(f"   {MCP_LOCAL_ENDPOINT}")
print("\n🔧 Use this URL to configure GitHub Copilot in the next step.")

📡 Local MCP Server Endpoint:
   http://localhost:7071/runtime/webhooks/mcp

🔧 Use this URL to configure GitHub Copilot in the next step.


## Step 7: Configure GitHub Copilot to Use Your MCP Server

Now we connect VS Code's Copilot to your running MCP server. This allows Copilot to discover and invoke your custom tools.

### MCP Configuration Flow

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│   VS Code       │     │  .vscode/       │     │  MCP Server     │
│   Copilot       │────►│  mcp.json       │────►│  localhost:7071 │
│   Agent Mode    │     │  (config)       │     │  /webhooks/mcp  │
└─────────────────┘     └─────────────────┘     └─────────────────┘
```

### Add MCP Server via Command Palette

1. Press `F1` → **MCP: Add Server**
2. Transport type: **HTTP (Server-Sent Events)**
3. URL: `http://localhost:7071/runtime/webhooks/mcp`
4. Scope: **Workspace** (saves to `.vscode/mcp.json`)

### Verify Connection

1. `F1` → **MCP: List Servers** — Confirm server is listed
2. Open **Copilot Chat** → Switch to **Agent Mode**
3. The tools panel should show your registered tools

### Test Prompts

Try these in Copilot Chat (Agent Mode):

| Prompt | Expected Tool |
|--------|---------------|
| "Say hello" | `hello_mcp` |
| "Analyze these numbers: [10, 20, 30, 40, 50]" | `analyze_data` |
| "Save this as my_snippet: `print('hello')`" | `save_snippet` |

In [13]:
# MCP Configuration for VS Code (.vscode/mcp.json)
mcp_config = """
{
    "servers": {
        "lab4-mcp-server": {
            "type": "http",
            "url": "http://localhost:7071/runtime/webhooks/mcp"
        }
    }
}
"""

# Create .vscode directory and save the config
vscode_dir = PROJECT_ROOT / ".vscode"
vscode_dir.mkdir(parents=True, exist_ok=True)

mcp_config_path = vscode_dir / "mcp.json"
with open(mcp_config_path, "w") as f:
    f.write(mcp_config.strip())

print(f"✅ Created MCP configuration: {mcp_config_path}")
print("\n📄 Configuration:")
print(mcp_config)

✅ Created MCP configuration: ../src/tool_registry/mcps/.vscode/mcp.json

📄 Configuration:

{
    "servers": {
        "lab4-mcp-server": {
            "type": "http",
            "url": "http://localhost:7071/runtime/webhooks/mcp"
        }
    }
}



## Step 8: Deploy to Azure

Deploying to Azure gives you a production-ready, scalable MCP server accessible from anywhere.

### ⚠️ IMPORTANT: Choose a Unique Environment Name

Since everyone in class is deploying to Azure, you **MUST use a unique environment name** to avoid conflicts. The environment name (`-e` flag) determines your resource group and function app names.

**Naming Convention:** Use `mcp-<your-name>` or `mcp-<your-netid>`:

```bash
# ✅ Good - unique names
azd up -e mcp-johndoe
azd up -e mcp-jsmith42
azd up -e mcp-pablo

# ❌ Bad - will conflict with others
azd up -e mcpserver-lab4
azd up -e mcp-server
azd up -e my-mcp
```

> **💡 Tip**: If you already initialized with `azd init -e <name>`, you can change your environment name by running `azd env select <new-name>` or starting fresh with `azd env new mcp-<your-name>`.

### Interactive Prompts During Deployment

When you run `azd up`, you'll be asked several questions:

| Prompt | What to Do |
|--------|-----------|
| **Select an Azure Subscription** | Choose your subscription (e.g., `Azure 2 for MSAI` for class). Use arrow keys and type to filter. |
| **Enter a value for 'location'** | Pick a region close to you. **East US** is recommended, but if you see capacity errors, try **East US 2**, **West US 2**, or **Central US**. |
| **Enter a value for 'vnetEnabled'** | Select **False** (we don't need a virtual network for this lab). |

**Example:**
```
? Select an Azure Subscription to use: 15. Azure 2 for MSAI (3cc20dd6-...)
? Enter a value for the 'location' infrastructure parameter: 24. (US) East US (eastus)
? Enter a value for the 'vnetEnabled' infrastructure parameter: False
```

> **💡 Capacity Errors?** If deployment fails with a capacity error in your chosen region, delete the resources and try a different region:
> ```bash
> azd down --purge  # Delete failed deployment
> azd up -e mcp-<your-name>  # Try again, pick different region
> ```

### Deployment Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                         Azure Cloud                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   ┌─────────────┐    ┌─────────────┐    ┌─────────────────┐    │
│   │  Function   │    │  Storage    │    │  Application    │    │
│   │  App (Flex) │◄──►│  Account    │    │  Insights       │    │
│   │             │    │             │    │  (Monitoring)   │    │
│   └──────┬──────┘    └─────────────┘    └─────────────────┘    │
│          │                                                       │
│          │ https://<app>.azurewebsites.net/runtime/webhooks/mcp │
│          │                                                       │
└──────────┼───────────────────────────────────────────────────────┘
           │
           ▼
    ┌─────────────┐
    │  AI Agents  │  (Copilot, custom agents, etc.)
    └─────────────┘
```

### Deploy with Azure Developer CLI

```bash
cd src/tool_registry/mcps

# Authenticate (opens browser)
azd auth login

# Provision infrastructure + deploy code
# ⚠️ Replace <your-name> with YOUR unique identifier!
azd up -e mcp-<your-name>
```

**What `azd up` does:**
1. **Provision**: Creates Resource Group (`rg-mcp-<your-name>`), Storage Account, Function App (Flex Consumption)
2. **Build**: Packages your Python code
3. **Deploy**: Pushes to Azure Functions

> ⚠️ **Note**: You may see a 503 error at the end of deployment. This is usually transient - the Function App needs a moment to initialize. Wait 30 seconds and proceed to retrieve your credentials.

### Retrieve Deployment Credentials

After `azd up` completes, run these commands in your terminal to get your credentials:

```bash
# Navigate to the MCP project
cd src/tool_registry/mcps

# Get environment values
azd env get-values

# Extract the function name and build the resource group name
AZURE_FUNCTION_NAME=$(azd env get-values | grep AZURE_FUNCTION_NAME | cut -d'=' -f2 | tr -d '"')
AZURE_ENV_NAME=$(azd env get-values | grep AZURE_ENV_NAME | cut -d'=' -f2 | tr -d '"')
RESOURCE_GROUP="rg-${AZURE_ENV_NAME}"

# Get the MCP extension key
MCP_KEY=$(az functionapp keys list \
    --resource-group $RESOURCE_GROUP \
    --name $AZURE_FUNCTION_NAME \
    --query "systemKeys.mcp_extension" -o tsv)

# Display your credentials
echo "============================================"
echo "🔗 MCP URL: https://${AZURE_FUNCTION_NAME}.azurewebsites.net/runtime/webhooks/mcp"
echo "🔑 MCP Key: ${MCP_KEY}"
echo "============================================"
```

**Expected Output:**
```
============================================
🔗 MCP URL: https://func-api-xxxxx.azurewebsites.net/runtime/webhooks/mcp
🔑 MCP Key: yourkey==
============================================
```

### Update MCP Configuration for Remote Server

Update your `.vscode/mcp.json` with the remote server credentials:

```json
{
    "servers": {
        "lab4-mcp-server-local": {
            "type": "http",
            "url": "http://localhost:7071/runtime/webhooks/mcp"
        },
        "lab4-mcp-server-remote": {
            "type": "http",
            "url": "https://<YOUR-FUNCTION-APP>.azurewebsites.net/runtime/webhooks/mcp",
            "headers": {
                "x-functions-key": "<YOUR-MCP-KEY>"
            }
        }
    }
}
```

**Or run this one-liner** to generate the config automatically:

```bash
cd src/tool_registry/mcps

AZURE_FUNCTION_NAME=$(azd env get-values | grep AZURE_FUNCTION_NAME | cut -d'=' -f2 | tr -d '"')
AZURE_ENV_NAME=$(azd env get-values | grep AZURE_ENV_NAME | cut -d'=' -f2 | tr -d '"')
MCP_KEY=$(az functionapp keys list --resource-group "rg-${AZURE_ENV_NAME}" --name $AZURE_FUNCTION_NAME --query "systemKeys.mcp_extension" -o tsv)

cat > .vscode/mcp.json << EOF
{
    "servers": {
        "lab4-mcp-server-local": {
            "type": "http",
            "url": "http://localhost:7071/runtime/webhooks/mcp"
        },
        "lab4-mcp-server-remote": {
            "type": "http",
            "url": "https://${AZURE_FUNCTION_NAME}.azurewebsites.net/runtime/webhooks/mcp",
            "headers": {
                "x-functions-key": "${MCP_KEY}"
            }
        }
    }
}
EOF

echo "✅ Updated .vscode/mcp.json"
cat .vscode/mcp.json
```

> 💡 **Tip**: After updating the config, run `F1` → **MCP: List Servers** to refresh.

## Step 9: Test Your Deployed MCP Server

### Quick Health Check

Verify the remote MCP server is responding:

```bash
# Set your credentials (from previous step)
AZURE_FUNCTION_NAME="func-api-xxxxx"  # Replace with your actual function name
MCP_KEY="your-mcp-key-here"           # Replace with your actual key

# Test the SSE endpoint
curl -s -X GET \
  "https://${AZURE_FUNCTION_NAME}.azurewebsites.net/runtime/webhooks/mcp/sse" \
  -H "x-functions-key: ${MCP_KEY}" \
  --max-time 5 | head -5
```

**Expected Output:**
```
event: endpoint
data: message?azmcpcs=...&code=...
```

If you see the `event: endpoint` response, your MCP server is live! 🎉

### Test MCP Server with Python

You can also test the MCP server programmatically using the `requests` library. This is useful for:
- Automated testing in CI/CD pipelines
- Building custom MCP clients
- Debugging tool responses

**Setup**: Create a `.env` file with your credentials (copy from `.env.sample`):

```bash
cd src/tool_registry/mcps
cp .env.sample .env
# Edit .env with your actual values
```

The `.env` file should contain:
```
MCP_REMOTE_URL=https://func-api-xxxxx.azurewebsites.net/runtime/webhooks/mcp
MCP_REMOTE_KEY=your-key-here
MCP_LOCAL_URL=http://localhost:7071/runtime/webhooks/mcp
```

> ⚠️ **Security**: The `.env` file is gitignored and should never be committed!

In [17]:
import requests
import json
import os
from dotenv import load_dotenv

# =============================================================================
# Load credentials from .env file
# =============================================================================

# Load environment variables from .env file
env_path = PROJECT_ROOT / ".env"
load_dotenv(env_path)

# Remote server credentials (loaded from .env)
MCP_REMOTE_URL = os.getenv("MCP_REMOTE_URL")
MCP_REMOTE_KEY = os.getenv("MCP_REMOTE_KEY")

# Local server (no key needed)
MCP_LOCAL_URL = os.getenv("MCP_LOCAL_URL", "http://localhost:7071/runtime/webhooks/mcp")

# Verify credentials loaded
if not MCP_REMOTE_URL or not MCP_REMOTE_KEY:
    print("⚠️ Missing credentials! Make sure .env file exists at:")
    print(f"   {env_path}")
    print("\n   Copy .env.sample to .env and fill in your values.")
else:
    print(f"✅ Loaded credentials from {env_path}")
    print(f"   URL: {MCP_REMOTE_URL}")
    print(f"   Key: {'*' * 10}...{MCP_REMOTE_KEY[-6:]}")

✅ Loaded credentials from ../src/tool_registry/mcps/.env
   URL: https://func-api-4csy4jcpcp47y.azurewebsites.net/runtime/webhooks/mcp
   Key: **********...kdfQ==


In [18]:
# =============================================================================
# Test: Verify MCP Server SSE Endpoint
# =============================================================================
# MCP uses Server-Sent Events (SSE) for real-time communication.
# We can verify the server is running by checking the SSE endpoint.

print("🧪 Testing MCP Server Connection")
print("=" * 50)


def test_mcp_sse(url: str, key: str = None, timeout: int = 15) -> dict:
    """Test the MCP server's SSE endpoint."""
    headers = {"Content-Type": "application/json"}
    if key:
        headers["x-functions-key"] = key

    sse_url = url.rstrip("/") + "/sse"

    try:
        # Connect to SSE endpoint (stream=True for SSE)
        response = requests.get(sse_url, headers=headers, timeout=timeout, stream=True)

        # Read the first event
        first_line = next(response.iter_lines(), b"").decode()
        second_line = next(response.iter_lines(), b"").decode()
        response.close()

        return {
            "status": "healthy" if response.status_code == 200 else "error",
            "status_code": response.status_code,
            "event": first_line,
            "data": second_line,
        }
    except requests.exceptions.Timeout:
        return {
            "status": "timeout",
            "error": f"Server did not respond within {timeout}s (cold start?)",
        }
    except requests.exceptions.RequestException as e:
        return {"status": "error", "error": str(e)}


# Test remote server
print(f"\n📡 Remote Server: {MCP_REMOTE_URL.split('//')[1].split('.')[0]}")
result = test_mcp_sse(MCP_REMOTE_URL, MCP_REMOTE_KEY)

if result["status"] == "healthy":
    print(f"   ✅ Status: HTTP {result['status_code']}")
    print(f"   📨 Event: {result.get('event', 'N/A')}")
    print(f"   📦 Data: {result.get('data', 'N/A')[:80]}...")
else:
    print(f"   ❌ Status: {result['status']}")
    print(f"   Error: {result.get('error', 'Unknown')}")
    print("\n   💡 Tip: Azure Functions may need 15-30s to cold start. Try again!")

🧪 Testing MCP Server Connection

📡 Remote Server: func-api-4csy4jcpcp47y
   ✅ Status: HTTP 200
   📨 Event: event: endpoint
   📦 Data: ...


In [19]:
# =============================================================================
# Test: Local Server (if running)
# =============================================================================

print("🧪 Testing Local MCP Server")
print("=" * 50)

print(f"\n📡 Local Server: localhost:7071")
local_result = test_mcp_sse(MCP_LOCAL_URL, timeout=5)

if local_result["status"] == "healthy":
    print(f"   ✅ Status: HTTP {local_result['status_code']}")
    print(f"   📨 Event: {local_result.get('event', 'N/A')}")
else:
    print(f"   ⚠️ Status: {local_result['status']}")
    print(f"   (This is expected if you're not running 'func start' locally)")

🧪 Testing Local MCP Server

📡 Local Server: localhost:7071
   ✅ Status: HTTP 200
   📨 Event: event: endpoint


### Why Can't We Call MCP Tools Directly with Python?

**MCP uses Server-Sent Events (SSE)**, not a simple REST API. Here's the communication flow:

```
┌─────────────────┐                    ┌─────────────────┐
│   MCP Client    │                    │   MCP Server    │
│   (Copilot)     │                    │ (Azure Function)│
├─────────────────┤                    ├─────────────────┤
│ 1. Connect SSE  │ ─────────────────► │                 │
│                 │ ◄───────────────── │ 2. Send tools   │
│ 3. User prompt  │                    │    metadata     │
│ 4. Call tool    │ ─────────────────► │                 │
│                 │ ◄───────────────── │ 5. Return result│
│ 6. Display      │                    │                 │
└─────────────────┘                    └─────────────────┘
```

The MCP protocol requires:
1. **Persistent SSE connection** for real-time messaging
2. **JSON-RPC 2.0** format for requests/responses
3. **Tool discovery** before invocation

**To test MCP tools, use GitHub Copilot in Agent Mode** — it handles all the protocol complexity for you.

### Test in GitHub Copilot

1. **Refresh MCP Servers**: Press `F1` → **MCP: List Servers**
2. **Open Copilot Chat**: Click the Copilot icon or press `Ctrl+Shift+I`
3. **Switch to Agent Mode**: Click the dropdown and select **Agent**
4. **Look for your tools**: You should see `lab4-mcp-server-remote` with tools:
   - `hello_mcp`
   - `analyze_data`
   - `save_snippet`
   - `get_snippet`
   - `list_snippets`

**Test Prompts** (try these in Agent Mode):

| Prompt | Expected Result |
|--------|-----------------|
| "Say hello using the MCP server" | Returns greeting from `hello_mcp` |
| "Analyze [100, 200, 300, 400, 500]" | Returns statistics from `analyze_data` |
| "What's the mean of 10, 20, 30?" | Copilot calls `analyze_data` and interprets results |

## 🎯 Summary

### What You Built

You created an **end-to-end MCP server** that:
- Exposes Python functions as **AI-consumable tools**
- Runs locally with **Azure Functions Core Tools**
- Deploys to **Azure** with a single command
- Integrates with **GitHub Copilot** via the MCP protocol

### Key Takeaways

| Concept | Implementation |
|---------|----------------|
| **Tool Definition** | `@app.generic_trigger(type="mcpToolTrigger", ...)` |
| **Parameter Schema** | `toolProperties` JSON with name, type, description |
| **Local Dev** | Azurite (storage) + `func start` (runtime) |
| **Remote Deploy** | `azd up` → Flex Consumption Function App |
| **Security** | Function keys (`x-functions-key` header) |

### From Lab 1 → Lab 4: The Journey

```
Lab 1: HTTP Function          Lab 4: MCP Server
┌─────────────────┐          ┌─────────────────┐
│ analyze_data()  │   ───►   │ analyze_data()  │
│ HTTP Trigger    │          │ MCP Tool Trigger│
│ Manual REST call│          │ AI Agent invokes│
└─────────────────┘          └─────────────────┘
```

**The same business logic, now accessible to AI agents.**

## 📚 Additional Resources

- [MCP Specification](https://spec.modelcontextprotocol.io/) — Protocol details
- [Azure Functions MCP Extension](https://learn.microsoft.com/en-us/azure/azure-functions/functions-bindings-mcp) — Binding reference
- [Azure Developer CLI](https://learn.microsoft.com/en-us/azure/developer/azure-developer-cli/) — Deployment tool
- [GitHub Copilot MCP](https://docs.github.com/en/copilot/using-github-copilot/using-mcp-servers) — Client configuration

---

**Congratulations!** 🎉 You've built an MCP server that bridges your business logic with AI agents.